# TINT Demonstration


In [ ]:
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, Image, display
import tempfile
import os
import shutil

import pyart
from tint.data_utils import get_nexrad_keys, read_nexrad_key
from tint import Cell_tracks, animate

## Data


In [ ]:
# Let's get some nexrad data from Amazon S3
keys = get_nexrad_keys('khgx', start='20150704_121000', end='20150704_122500')

In [ ]:
# We can visualize this data using pyart
fig = plt.figure(figsize=(10, 7))


def plot_ppi(key):
    plt.clf()
    radar = read_nexrad_key(key)
    display = pyart.graph.RadarMapDisplay(radar)
    display.plot_ppi_map('reflectivity', resolution='l',
                         sweep=3, fig=fig, width=200000, height=200000)
    del display, radar


ppi_anim = FuncAnimation(fig, plot_ppi, frames=keys, interval=1000)
HTML(ppi_anim.to_jshtml())

### Disclaimer
If you don't want this data to remain on your filesystem, make sure you run the last cell in this notebook. If your kernel crashes for some reason, you can manually remove the the path printed after the following cell.

In [ ]:
# These radar volumes look interesting.
# Let's make a function that maps them to a cartesian grid using pyart.


def get_grid(radar):
    """ Returns grid object from radar object. """
    grid = pyart.map.grid_from_radars(
        radar, grid_shape=(20, 200, 200),
        grid_limits=((0, 10000), (-100000, 0), (0, 100000)),
        fields=['reflectivity'], gridding_algo='map_gates_to_grid')
    return grid


# Let's write these grids to a temporary location so that we can reuse them.
tmp_dir = tempfile.mkdtemp()
print('temporary directory:', tmp_dir)
filenames = []
for num, key in enumerate(keys):
    print('saving grid', num)
    radar = read_nexrad_key(key)
    grid = get_grid(radar)
    name = os.path.join(tmp_dir, 'grid_' + str(num).zfill(3) + '.nc')
    filenames.append(name)
    pyart.io.write_grid(name, grid)
    del radar, grid

In [ ]:
grids = (pyart.io.read_grid(fn) for fn in filenames)

## Let's do some tracking

In [ ]:
tracks_obj = Cell_tracks()
tracks_obj.params

In [ ]:
tracks_obj.get_tracks(grids)

In [ ]:
tracks_obj.tracks

In [ ]:
# lets make another generator for the animator
grids = (pyart.io.read_grid(fn) for fn in filenames)

In [ ]:
animate(tracks_obj, grids, os.path.join(tmp_dir, 'tint_demo_anim'))

## Clean Up

In [ ]:
shutil.rmtree(tmp_dir)